Import packages

In [1]:
import numpy as np
from RL import Agent
# from utils import plot_learning_curve
from env import env
import random
import os

function to create directory

In [2]:
parent_dir_path = "/Users/ashu/Projects/python/wsn/results/"

def dir():
	parent_dir = parent_dir_path
	filename = 'result'
	path = os.path.join(parent_dir,filename)
	try:
		os.makedirs(path)
	except FileExistsError:
		filename += str(random.randint(0,1000))
		path = os.path.join(parent_dir, filename)
		os.makedirs(path)
	print("result path:", path)
	return path


function to write history

In [3]:
def write_history(par_path, history, round):
	fname = "history"
	if(round != -1):
		fname += str(round)
	fname += ".txt"
	path = os.path.join(par_path,  fname)
	with open(path, "w") as fp:
		fp.write("no.,acc,apl,small_worldness\n")
		for i in range(len(history)):
			s = str(i) + "," + str(history[i][0]) + "," + str(history[i][1]) + "," + str(history[i][2])+ "\n"
			fp.write(s)

setting parameters, environments and agent

In [4]:
N = 5
batch_size = 5
n_epochs = 4
alpha = 0.0003
env1 = env()
env1.set_nxg()
acc = env1.acc
apl =  env1.apl
best_apl = apl
best_acc = acc
n_games = 20
number_of_edges = 200
history = []

agent = Agent(n_actions=env1.action_space_n, batch_size=batch_size,
				alpha=alpha, n_epochs=n_epochs,
				input_dims=[env1.get_observation_space_shape()])
path = dir()
print(env1.action_space_n)
env1.save_graph(path, -1)

best_score = 0
score_history = []
learn_iters = 0
avg_score = 0
n_steps = 0

{'area_x': 500, 'area_y': 500, 'number_of_nodes': 400, 'base_x': 0, 'base_y': 0, 'node_initial_energy': 1, 'node_critical_energy': 0.4, 'dist_para': 2000, 'len_of_packets': 200, 'transmission_rate': 2000, 'speed_of_transmission': 300000000.0, 'radio_distance': 50}
result path: /Users/ashu/Projects/python/wsn/results/result401
77624


<Figure size 1200x800 with 0 Axes>

Running the model for n_games epochs.  
every epoch, we add 200 nodes & then reset the environment

In [5]:
env1.reset()
observation_flatten = env1.flatten_obs_sp()
edges_added_this_round= set()
score = 0
done = 0
rnd_his = []
rnd_his.append([env1.acc, env1.apl, round(env1.acc/env1.apl, 4)])

for i in range(number_of_edges):
	tempEnv = env1
	tempEnv.action_space = env1.action_space
	tempEnv.obs_space = env1.obs_space
	tempEnv.action_space_n = env1.action_space_n
	tempEnv.nxg  = env1.nxg
	tempEnv.init_nxg = env1.nxg
	tempEnv.initial_graph = env1.graph
	tempEnv.initial_net = env1.net
	
	while n_games < 30 and len(edges_added_this_round) > 1:
		tempEnv.reset()
		observation_flatten = tempEnv.flatten_obs_sp()
		edges_added_this_round= set()
		score = 0
		done = 0
		rnd_his = []
		rnd_his.append([tempEnv.acc, tempEnv.apl, round(tempEnv.acc/tempEnv.apl, 4)])
		
		while not done == number_of_edges:
			print("round : ", done)
			action, prob, val = agent.choose_action(observation_flatten)
			# observation_, reward, done, info = env.step(action)
			observation_, reward = tempEnv.step(action)
			edges_added_this_round.add(action)											## commment later
			rnd_his.append([tempEnv.acc, tempEnv.apl, round(tempEnv.acc/tempEnv.apl, 4)])
			n_steps += 1
			score += reward
			done += 1																			## comment later
			agent.remember(observation_, action, prob, val, reward, done)
			if n_steps % N == 0:
				agent.learn()
				learn_iters += 1
			observation_flatten = observation_
			print("-------")
		agent.learn()
		write_history(path, rnd_his, i)
		score_history.append(score)
		avg_score = np.mean(score_history[-100:])
		if avg_score > best_score:
			best_score = avg_score
		if best_acc < tempEnv.acc or best_apl > tempEnv.apl:
			best_acc = tempEnv.acc
			best_apl = tempEnv.apl

		history.append([tempEnv.acc, tempEnv.apl, round(tempEnv.acc/tempEnv.apl, 4)])
		tempEnv.save_graph(path, i)
		tempEnv.save_graph_npy(path, i)
		print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
				'time_steps', n_steps, 'learning_steps', learn_iters)
		
	for edge in edges_added_this_round:
		env1.set_step_progess(edge)
		agent = Agent(n_actions=env1.action_space_n, batch_size=batch_size,
				alpha=alpha, n_epochs=n_epochs,
				input_dims=[env1.get_observation_space_shape()])
	
	env1.save_graph(path + "/env/", i)
	env1.save_graph_npy(path + "/env/", i)
	

{'area_x': 500, 'area_y': 500, 'number_of_nodes': 400, 'base_x': 0, 'base_y': 0, 'node_initial_energy': 1, 'node_critical_energy': 0.4, 'dist_para': 2000, 'len_of_packets': 200, 'transmission_rate': 2000, 'speed_of_transmission': 300000000.0, 'radio_distance': 50}
graph .npy saved : /Users/ashu/Projects/python/wsn/results/result401/env/
{'area_x': 500, 'area_y': 500, 'number_of_nodes': 400, 'base_x': 0, 'base_y': 0, 'node_initial_energy': 1, 'node_critical_energy': 0.4, 'dist_para': 2000, 'len_of_packets': 200, 'transmission_rate': 2000, 'speed_of_transmission': 300000000.0, 'radio_distance': 50}
graph .npy saved : /Users/ashu/Projects/python/wsn/results/result401/env/
{'area_x': 500, 'area_y': 500, 'number_of_nodes': 400, 'base_x': 0, 'base_y': 0, 'node_initial_energy': 1, 'node_critical_energy': 0.4, 'dist_para': 2000, 'len_of_packets': 200, 'transmission_rate': 2000, 'speed_of_transmission': 300000000.0, 'radio_distance': 50}
graph .npy saved : /Users/ashu/Projects/python/wsn/result

KeyboardInterrupt: 

<Figure size 1200x800 with 0 Axes>